# Advanced Querying Mongo

Importing libraries and setting up connection

In [1]:
# !pip install pymongo
from pymongo import MongoClient
client = MongoClient("mongodb://localhost:27017")
db= client ['Companies']
collection = db['Companies']

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [2]:
# Your Code 

query= {"name": "Babelgum"}
select = {"_id":0, "name":1}

list(collection.find(query, select))
 


[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [3]:
# Your Code
query = {"number_of_employees": {"$gte" : 5000}}

list(collection.find(query).sort("number_of_employees", 1).limit(1))


[{'_id': ObjectId('52cdef7d4bab8bd67529891b'),
  'name': 'Spotify',
  'permalink': 'spotify',
  'crunchbase_url': 'http://www.crunchbase.com/company/spotify',
  'homepage_url': 'http://www.spotify.com',
  'blog_url': 'http://www.spotify.com/blog/',
  'blog_feed_url': 'http://www.spotify.com/blog/feed',
  'twitter_username': 'spotify',
  'category_code': 'music',
  'number_of_employees': 5000,
  'founded_year': 2006,
  'founded_month': None,
  'founded_day': None,
  'deadpooled_year': None,
  'deadpooled_month': None,
  'deadpooled_day': None,
  'deadpooled_url': None,
  'tag_list': 'music-streaming, streaming, music, cloud',
  'alias_list': '',
  'email_address': 'press@spotify.com',
  'phone_number': '0709821244',
  'description': 'Music streaming over the internet',
  'created_at': 'Mon Apr 28 19:10:19 UTC 2008',
  'updated_at': 'Thu Dec 12 00:45:40 UTC 2013',
  'overview': '<p>Spotify has created a lightweight software application that allows instant listening to specific tracks or 

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [4]:
# Your Code

query = {"$and": [{"founded_year": {"$lte":2005}}, {"founded_year": {"$gte" : 2000}}]}
select = {"_id":0, "name":1, "founded_year":1}

list(collection.find(query, select).limit(5))


[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Digg', 'founded_year': 2004},
 {'name': 'Facebook', 'founded_year': 2004},
 {'name': 'Omnidrive', 'founded_year': 2005}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [47]:
# Your Code
query = {"$and": [{"acquisition.price_amount": {"$gt":100000000}}, {"founded_year": {"$lt":2010}}]}
select = {"_id" :0, "name": 1, "acquisition.price_amount": 1}

list(collection.find(query, select).limit(5))

[{'name': 'Postini', 'acquisition': {'price_amount': 625000000}},
 {'name': 'Plaxo', 'acquisition': {'price_amount': 150000000}},
 {'name': 'Jajah', 'acquisition': {'price_amount': 207000000}},
 {'name': 'Skype', 'acquisition': {'price_amount': 2600000000.0}},
 {'name': 'YouTube', 'acquisition': {'price_amount': 1650000000}}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [6]:
# Your Code

# Your Code

query = {"$and": [{"number_of_employees": {"$lt":1000}}, {"founded_year": {"$lt" : 2005}}]}
select = {"_id":0, "name":1, "founded_year":1}

list(collection.find(query, select).sort("number_of_employees", 1).limit(10))


[{'name': 'Fox Interactive Media', 'founded_year': 1979},
 {'name': 'Ticketmaster', 'founded_year': 1976},
 {'name': 'Eurekster', 'founded_year': 2004},
 {'name': 'MindTouch', 'founded_year': 2004},
 {'name': 'Skype', 'founded_year': 2003},
 {'name': 'Monster', 'founded_year': 1994},
 {'name': 'EditGrid', 'founded_year': 2003},
 {'name': 'Simpy', 'founded_year': 2004},
 {'name': 'Compete', 'founded_year': 2000},
 {'name': 'stylediary', 'founded_year': 2004}]

### 6. All the companies that don't include the `partners` field.

In [58]:
# Your Code

# Your Code

query = { "partners": {"$exists" :False}}


list(collection.find(query).limit(1))


[]

### 7. All the companies that have a null type of value on the `category_code` field.

In [8]:
# Your Code
# Your Code

query = { "category_code": {"$eq" : None}}

list(collection.find(query).limit(1))



[{'_id': ObjectId('52cdef7c4bab8bd6752980f6'),
  'name': 'Collective',
  'permalink': 'collective',
  'crunchbase_url': 'http://www.crunchbase.com/company/collective',
  'homepage_url': None,
  'blog_url': None,
  'blog_feed_url': None,
  'twitter_username': None,
  'category_code': None,
  'number_of_employees': None,
  'founded_year': None,
  'founded_month': None,
  'founded_day': None,
  'deadpooled_year': None,
  'deadpooled_month': None,
  'deadpooled_day': None,
  'deadpooled_url': None,
  'tag_list': None,
  'alias_list': None,
  'email_address': None,
  'phone_number': None,
  'description': None,
  'created_at': 'Thu Sep 26 13:15:02 UTC 2013',
  'updated_at': 'Thu Sep 26 13:15:02 UTC 2013',
  'overview': None,
  'image': None,
  'products': [],
  'relationships': [],
  'competitions': [],
  'providerships': [],
  'total_money_raised': '$0',
  'funding_rounds': [],
  'investments': [],
  'acquisition': None,
  'acquisitions': [],
  'offices': [],
  'milestones': [],
  'ipo': N

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [9]:
# Your Code
query = {"$and": [{"number_of_employees": {"$gte":100}}, {"number_of_employees": {"$lt":1000}}]}
select = {"_id":0, "name":1, "number_of_employees":1}

list(collection.find(query, select).limit(2))



[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120}]

### 9. Order all the companies by their IPO price in a descending order.

In [10]:
# Your Code
list(collection.find().sort("total_money_raised", -1).limit(1))

[{'_id': ObjectId('52cdef7e4bab8bd67529b747'),
  'name': 'Tuenti Technologies',
  'permalink': 'tuenti-technologies',
  'crunchbase_url': 'http://www.crunchbase.com/company/tuenti-technologies',
  'homepage_url': 'http://www.tuenti.com',
  'blog_url': 'http://blog.tuenti.com/',
  'blog_feed_url': 'http://blog.tuenti.com/feed/',
  'twitter_username': 'tuenti',
  'category_code': 'social',
  'number_of_employees': 170,
  'founded_year': 2006,
  'founded_month': 5,
  'founded_day': 1,
  'deadpooled_year': None,
  'deadpooled_month': None,
  'deadpooled_day': None,
  'deadpooled_url': None,
  'tag_list': 'social-network, spain',
  'alias_list': '',
  'email_address': 'zaryn@tuenti.com',
  'phone_number': '+34 914 294 039',
  'description': 'Social Network',
  'created_at': 'Wed Apr 15 15:07:37 UTC 2009',
  'updated_at': 'Tue Dec 10 05:33:09 UTC 2013',
  'overview': '<p>Tuenti is a Madrid-based, social networking website that has been referred to as the &#8220;Spanish Facebook.&#8221; Tuent

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [11]:
# Your Code
list(collection.find().sort("number_of_employees", 1).limit(10))

[{'_id': ObjectId('52cdef7c4bab8bd675297d8f'),
  'name': 'Omnidrive',
  'permalink': 'omnidrive',
  'crunchbase_url': 'http://www.crunchbase.com/company/omnidrive',
  'homepage_url': 'http://www.omnidrive.com',
  'blog_url': 'http://www.omnidrive.com/blog',
  'blog_feed_url': 'http://feeds.feedburner.com/omnidrive',
  'twitter_username': 'Nomadesk',
  'category_code': 'network_hosting',
  'number_of_employees': None,
  'founded_year': 2005,
  'founded_month': 11,
  'founded_day': 1,
  'deadpooled_year': 2008,
  'deadpooled_month': 9,
  'deadpooled_day': 15,
  'deadpooled_url': '',
  'tag_list': 'storage, sharing, edit, online',
  'alias_list': None,
  'email_address': 'info@omnidrive.com',
  'phone_number': '660-675-5052',
  'description': None,
  'created_at': 'Sun May 27 03:25:32 UTC 2007',
  'updated_at': 'Tue Jul 02 22:48:04 UTC 2013',
  'overview': '<p>Currently in public beta, Omnidrive makes it easy to access, edit and share your files from any computer with a web browser. The s

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [12]:
# Your Code
query = {"founded_month": {"$gte": 6}}

list(collection.find(query).limit(1))


[{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
  'name': 'Wetpaint',
  'permalink': 'abc2',
  'crunchbase_url': 'http://www.crunchbase.com/company/wetpaint',
  'homepage_url': 'http://wetpaint-inc.com',
  'blog_url': 'http://digitalquarters.net/',
  'blog_feed_url': 'http://digitalquarters.net/feed/',
  'twitter_username': 'BachelrWetpaint',
  'category_code': 'web',
  'number_of_employees': 47,
  'founded_year': 2005,
  'founded_month': 10,
  'founded_day': 17,
  'deadpooled_year': 1,
  'tag_list': 'wiki, seattle, elowitz, media-industry, media-platform, social-distribution-system',
  'alias_list': '',
  'email_address': 'info@wetpaint.com',
  'phone_number': '206.859.6300',
  'description': 'Technology Platform Company',
  'created_at': datetime.datetime(2007, 5, 25, 6, 51, 27),
  'updated_at': 'Sun Dec 08 07:15:44 UTC 2013',
  'overview': '<p>Wetpaint is a technology platform company that uses its proprietary state-of-the-art technology and expertise in social media to build and mon

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [39]:
# Your Code

query = {"$and": [{"founded_year": {"$lt":2000}}, {"acquisition.price_amount": {"$lt":10000000}}]}

list(collection.find(query).limit(1))


[{'_id': ObjectId('52cdef7c4bab8bd675297e37'),
  'name': 'Cyworld',
  'permalink': 'cyworld',
  'crunchbase_url': 'http://www.crunchbase.com/company/cyworld',
  'homepage_url': 'http://us.cyworld.com',
  'blog_url': '',
  'blog_feed_url': '',
  'twitter_username': None,
  'category_code': 'web',
  'number_of_employees': None,
  'founded_year': 1999,
  'founded_month': 7,
  'founded_day': 1,
  'deadpooled_year': None,
  'deadpooled_month': None,
  'deadpooled_day': None,
  'deadpooled_url': None,
  'tag_list': 'cyworld',
  'alias_list': '',
  'email_address': '',
  'phone_number': '',
  'description': 'Social Network',
  'created_at': 'Fri Jul 20 09:34:30 UTC 2007',
  'updated_at': 'Fri Mar 15 21:08:14 UTC 2013',
  'overview': '<p>South Korea&#8217;s most popular social networking site, Cyworld, has now brought a version of its site to the US.  Founded in 1999, Cyworld was bought in 2003 by SK Telecom, Korea&#8217;s primary provider of mobile telecommunications.  Since then Cyworld has 

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [40]:
# Your Code
query = {"acquisition.acquired_year": {"$gt":2010}}
select= {"_id": 0, "name": 1, "acquisition": 1}

list(collection.find(query, select).sort("acquisition.price_amount", 1).limit(2))


[{'name': 'Kyte',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
   'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
   'acquired_year': 2011,
   'acquired_month': 1,
   'acquired_day': 31,
   'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}},
 {'name': 'Geni',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2012/11/28/all-in-the-family-myheritage-buys-former-yammer-stablemate-geni-com-raises-25m/',
   'source_description': 'MyHeritage acquires Geni and $25M to build family tree of the whole world',
   'acquired_year': 2012,
   'acquired_month': 11,
   'acquired_day': 28,
   'acquiring_company': {'name': 'MyHeritage', 'permalink': 'myheritage'}}}]

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [56]:
# Your Code
select= {"_id": 0, "name": 1, "founded_year": 1}

list(collection.find({},select).sort("founded_year", 1).limit(2))




[{'name': 'Lala', 'founded_year': None},
 {'name': 'Flektor', 'founded_year': None}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [44]:
# Your Code
query = {"founded_day": {"$lte":7}}

list(collection.find(query, select).sort("acquisition.price_amount", -1).limit(1))

[{'name': 'Netscape',
  'acquisition': {'price_amount': 4200000000.0,
   'price_currency_code': 'USD',
   'term_code': 'stock',
   'source_url': 'http://news.cnet.com/2100-1023-218360.html',
   'source_description': 'AOL buys Netscape for $4.2 billion',
   'acquired_year': 1998,
   'acquired_month': 11,
   'acquired_day': 28,
   'acquiring_company': {'name': 'AOL', 'permalink': 'aol'}}}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [33]:
# Your Code
query = {"$and": [{"homepage_url": {"$exists":True}}, {"number_of_employees": {"$gt":4000}}]}


list(collection.find().sort("number_of_employees", -1).limit(2))


[{'_id': ObjectId('52cdef7d4bab8bd67529941a'),
  'name': 'Siemens',
  'permalink': 'siemens',
  'crunchbase_url': 'http://www.crunchbase.com/company/siemens',
  'homepage_url': 'http://www.siemens.com',
  'blog_url': '',
  'blog_feed_url': '',
  'twitter_username': 'Siemens',
  'category_code': 'hardware',
  'number_of_employees': 405000,
  'founded_year': 1847,
  'founded_month': None,
  'founded_day': None,
  'deadpooled_year': None,
  'deadpooled_month': None,
  'deadpooled_day': None,
  'deadpooled_url': None,
  'tag_list': 'automation, building-technologies, drive-technology, energy',
  'alias_list': '',
  'email_address': 'contact@siemens.com',
  'phone_number': '49 89 636 34134',
  'description': 'Electronics and Electrical Engineering',
  'created_at': 'Thu Jul 31 09:29:43 UTC 2008',
  'updated_at': 'Thu Nov 28 20:32:55 UTC 2013',
  'overview': '<p>Siemens AG, an electronics and electrical engineering company, operates in the industry, energy, and healthcare sectors worldwide. 

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [45]:
# Your Code


query = {
    "$and": [
        {"acquisition.price_amount": {"$gt": 10000000}},
        {"acquisition.price_currency_code": {"$exists": "EUR"}}
    ]
}


list(collection.find(query).limit(1))

[{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
  'name': 'Wetpaint',
  'permalink': 'abc2',
  'crunchbase_url': 'http://www.crunchbase.com/company/wetpaint',
  'homepage_url': 'http://wetpaint-inc.com',
  'blog_url': 'http://digitalquarters.net/',
  'blog_feed_url': 'http://digitalquarters.net/feed/',
  'twitter_username': 'BachelrWetpaint',
  'category_code': 'web',
  'number_of_employees': 47,
  'founded_year': 2005,
  'founded_month': 10,
  'founded_day': 17,
  'deadpooled_year': 1,
  'tag_list': 'wiki, seattle, elowitz, media-industry, media-platform, social-distribution-system',
  'alias_list': '',
  'email_address': 'info@wetpaint.com',
  'phone_number': '206.859.6300',
  'description': 'Technology Platform Company',
  'created_at': datetime.datetime(2007, 5, 25, 6, 51, 27),
  'updated_at': 'Sun Dec 08 07:15:44 UTC 2013',
  'overview': '<p>Wetpaint is a technology platform company that uses its proprietary state-of-the-art technology and expertise in social media to build and mon

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [50]:
# Your Code
query = {"acquisition.acquired_month": {"$lte": 3}}
select = {"_id": 0, "name": 1, "acquisition" : 1}

list(collection.find(query, select).limit(1))


[{'name': 'Kyte',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
   'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
   'acquired_year': 2011,
   'acquired_month': 1,
   'acquired_day': 31,
   'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}}]

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [52]:
# Your Code

query = {"$and": 
         [
            {"founded_year": {"$gt": 2000}}, 
            {"founded_year" : {"$lt": 2010}}, 
            {"acquisition.acquired_year": {"$gte": 2011}}]}

list(collection.find(query).limit(1))

[{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
  'name': 'Wetpaint',
  'permalink': 'abc2',
  'crunchbase_url': 'http://www.crunchbase.com/company/wetpaint',
  'homepage_url': 'http://wetpaint-inc.com',
  'blog_url': 'http://digitalquarters.net/',
  'blog_feed_url': 'http://digitalquarters.net/feed/',
  'twitter_username': 'BachelrWetpaint',
  'category_code': 'web',
  'number_of_employees': 47,
  'founded_year': 2005,
  'founded_month': 10,
  'founded_day': 17,
  'deadpooled_year': 1,
  'tag_list': 'wiki, seattle, elowitz, media-industry, media-platform, social-distribution-system',
  'alias_list': '',
  'email_address': 'info@wetpaint.com',
  'phone_number': '206.859.6300',
  'description': 'Technology Platform Company',
  'created_at': datetime.datetime(2007, 5, 25, 6, 51, 27),
  'updated_at': 'Sun Dec 08 07:15:44 UTC 2013',
  'overview': '<p>Wetpaint is a technology platform company that uses its proprietary state-of-the-art technology and expertise in social media to build and mon

### 20. All the companies that have been 'deadpooled' after the third year.

In [55]:
# Your Code
query = {"deadpooled_year" : {"$gte": 3}}

list(collection.find(query).limit(1))

[{'_id': ObjectId('52cdef7c4bab8bd675297d8c'),
  'name': 'Zoho',
  'permalink': 'abc4',
  'crunchbase_url': 'http://www.crunchbase.com/company/zoho',
  'homepage_url': 'http://zoho.com',
  'blog_url': 'http://blogs.zoho.com/',
  'blog_feed_url': 'http://blogs.zoho.com/feed',
  'twitter_username': 'zoho',
  'category_code': 'software',
  'number_of_employees': 1600,
  'founded_year': 2005,
  'founded_month': 9,
  'founded_day': 15,
  'deadpooled_year': 3,
  'tag_list': 'zoho, officesuite, spreadsheet, writer, projects, sheet, crm, show, creator, wiki, planner, suite, notebook, chat, meeting, mail',
  'alias_list': '',
  'email_address': 'info@zohocorp.com',
  'phone_number': '1-888-204-3539',
  'description': 'Online Business Apps Suite',
  'created_at': 'Fri May 25 19:30:28 UTC 2007',
  'updated_at': 'Wed Oct 30 00:07:05 UTC 2013',
  'overview': '<p>Zoho offers a suite of Business, Collaboration &amp; Productivity applications. Apps include CRM, Customer Support, Office Suite, Email Ho